# Modelización predictiva de la pobreza educativa en hogares argentinos mediante aprendizaje automático y la ecuación de Mincer

## 1. Configuracion

In [1]:
# Instalar librerias
!pip install pyeph > /dev/null 2>&1

In [2]:
# Importar librerias
import pyeph #https://github.com/institutohumai/pyeph
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

## 2. Carga de datos

In [3]:
# Definir los años y trimestres disponibles
years = range(2017, 2024)  # Desde 2017 hasta 2023 inclusive
quarters = [1, 2, 3, 4]    # Cuatro trimestres por año

# Diccionarios para almacenar los DataFrames
eph_individuals = {}
eph_households = {}

for year in years:
    for quarter in quarters:
        try:
            # Descargar y almacenar la base de datos de individuos
            eph_ind = pyeph.get(data="eph", year=year, period=quarter, base_type='individual')
            eph_individuals[f'eph_{year}_t{quarter}'] = eph_ind
            print(f'Datos de individuos del año {year}, trimestre {quarter} descargados exitosamente.')
        except Exception as e:
            print(f'No se pudieron descargar los datos de individuos del año {year}, trimestre {quarter}. Error: {e}')

        try:
            # Descargar y almacenar la base de datos de hogares
            eph_hog = pyeph.get(data="eph", year=year, period=quarter, base_type='hogar')
            eph_households[f'eph_{year}_t{quarter}'] = eph_hog
            print(f'Datos de hogares del año {year}, trimestre {quarter} descargados exitosamente.')
        except Exception as e:
            print(f'No se pudieron descargar los datos de hogares del año {year}, trimestre {quarter}. Error: {e}')

# Verificar las claves de los diccionarios para confirmar las bases descargadas
print("Bases de datos de individuos descargadas:")
print(eph_individuals.keys())
print("Bases de datos de hogares descargadas:")
print(eph_households.keys())


Obtenido con exito: base_individual_2017T1.zip 
Datos de individuos del año 2017, trimestre 1 descargados exitosamente.

Obtenido con exito: base_hogar_2017T1.zip 
Datos de hogares del año 2017, trimestre 1 descargados exitosamente.

Obtenido con exito: base_individual_2017T2.zip 
Datos de individuos del año 2017, trimestre 2 descargados exitosamente.

Obtenido con exito: base_hogar_2017T2.zip 
Datos de hogares del año 2017, trimestre 2 descargados exitosamente.

Obtenido con exito: base_individual_2017T3.zip 
Datos de individuos del año 2017, trimestre 3 descargados exitosamente.

Obtenido con exito: base_hogar_2017T3.zip 
Datos de hogares del año 2017, trimestre 3 descargados exitosamente.

Obtenido con exito: base_individual_2017T4.zip 
Datos de individuos del año 2017, trimestre 4 descargados exitosamente.

Obtenido con exito: base_hogar_2017T4.zip 
Datos de hogares del año 2017, trimestre 4 descargados exitosamente.

Obtenido con exito: base_individual_2018T1.zip 
Datos de indivi

In [4]:
eph_individuals['eph_2023_t4']

,CODUSU,ANO4,TRIMESTRE,NRO_HOGAR,COMPONENTE,H15,REGION,MAS_500,AGLOMERADO,PONDERA,...,PDECIFR,ADECIFR,IPCF,DECCFR,IDECCFR,RDECCFR,GDECCFR,PDECCFR,ADECCFR,PONDIH
0,TQRMNOSUWHLMKQCDEFNFF00852219,2023,4,1,1,1,41,N,7,370,...,12.0,12,0,12,12.0,12,NaN,12.0,12,0
1,TQRMNOSUWHLMKQCDEFNFF00852219,2023,4,1,2,1,41,N,7,370,...,12.0,12,0,12,12.0,12,NaN,12.0,12,0
2,TQRMNOTXQHKOLMCDEGNFJ00794590,2023,4,1,1,1,44,N,17,281,...,9.0,7,"216666,67",8,8.0,7,NaN,8.0,6,387
3,TQRMNOTXQHKOLMCDEGNFJ00794590,2023,4,1,2,1,44,N,17,281,...,9.0,7,"216666,67",8,8.0,7,NaN,8.0,6,387
4,TQRMNOTXQHKOLMCDEGNFJ00794590,2023,4,1,3,0,44,N,17,281,...,9.0,7,"216666,67",8,8.0,7,NaN,8.0,6,387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47332,TQRMNORXTHLOLSCDEGJBF00853031,2023,4,1,1,1,43,S,13,634,...,NaN,4,65000,2,2.0,2,2.0,NaN,2,777
47333,TQRMNORXTHLOLSCDEGJBF00853031,2023,4,1,2,1,43,S,13,634,...,NaN,4,65000,2,2.0,2,2.0,NaN,2,777
47334,TQRMNORXTHLOLSCDEGJBF00853031,2023,4,1,3,1,43,S,13,634,...,NaN,4,65000,2,2.0,2,2.0,NaN,2,777
47335,TQRMNORXTHLOLSCDEGJBF00853031,2023,4,1,4,1,43,S,13,634,...,NaN,4,65000,2,2.0,2,2.0,NaN,2,777


In [7]:
# Verificar los títulos de las columnas del DataFrame eph_households['eph_2023_t4']
columnas = eph_households['eph_2023_t4'].columns
columnas_lista_h = eph_households['eph_2023_t4'].columns.tolist()

# Verificar los títulos de las columnas del DataFrame eph_individuals['eph_2023_t4']
columnas = eph_individuals['eph_2023_t4'].columns
columnas_lista_i = eph_individuals['eph_2023_t4'].columns.tolist()

print(columnas_lista_h)
print(columnas_lista_i)

['CODUSU', 'ANO4', 'TRIMESTRE', 'NRO_HOGAR', 'REALIZADA', 'REGION', 'MAS_500', 'AGLOMERADO', 'PONDERA', 'IV1', 'IV1_ESP', 'IV2', 'IV3', 'IV3_ESP', 'IV4', 'IV5', 'IV6', 'IV7', 'IV7_ESP', 'IV8', 'IV9', 'IV10', 'IV11', 'IV12_1', 'IV12_2', 'IV12_3', 'II1', 'II2', 'II3', 'II3_1', 'II4_1', 'II4_2', 'II4_3', 'II5', 'II5_1', 'II6', 'II6_1', 'II7', 'II7_ESP', 'II8', 'II8_ESP', 'II9', 'V1', 'V2', 'V21', 'V22', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19_A', 'V19_B', 'IX_TOT', 'IX_MEN10', 'IX_MAYEQ10', 'ITF', 'DECIFR', 'IDECIFR', 'RDECIFR', 'GDECIFR', 'PDECIFR', 'ADECIFR', 'IPCF', 'DECCFR', 'IDECCFR', 'RDECCFR', 'GDECCFR', 'PDECCFR', 'ADECCFR', 'PONDIH', 'VII1_1', 'VII1_2', 'VII2_1', 'VII2_2', 'VII2_3', 'VII2_4']
['CODUSU', 'ANO4', 'TRIMESTRE', 'NRO_HOGAR', 'COMPONENTE', 'H15', 'REGION', 'MAS_500', 'AGLOMERADO', 'PONDERA', 'CH03', 'CH04', 'CH05', 'CH06', 'CH07', 'CH08', 'CH09', 'CH10', 'CH11', 'CH12', 'CH13', 'CH14', 'CH15', 'CH15_

## 3. Analisis de corte transversal

### 3.1 Cuatro trimestre - 2023

In [35]:
# 1. Preparar los datos
df = eph_individuals['eph_2023_t4'].copy()

# Definir los valores permitidos para cada columna
valores_permitidos_region = [41, 44, 40, 42, 43, 1]
valores_permitidos_ch04 = [1, 2]
valores_permitidos_nivel_ed = [6, 4, 1, 2, 3, 5, 7]
valores_permitidos_ch07 = [2, 1, 5, 4, 3]

# Filtrar el DataFrame para mantener solo las filas que cumplan todas las condiciones
df_filtrado = df[
    df['REGION'].isin(valores_permitidos_region) &
    df['CH04'].isin(valores_permitidos_ch04) &
    df['NIVEL_ED'].isin(valores_permitidos_nivel_ed) &
    df['CH07'].isin(valores_permitidos_ch07)
]

# Eliminar filas donde P21 sea negativo
df_filtrado = df_filtrado[df_filtrado['P21'] >= 0]

# Obtener y ordenar los valores únicos de menor a mayor
valores_unicos_region = np.sort(df_filtrado['REGION'].unique())
valores_unicos_ch04 = np.sort(df_filtrado['CH04'].unique())
valores_unicos_nivel_ed = np.sort(df_filtrado['NIVEL_ED'].unique())
valores_unicos_ch07 = np.sort(df_filtrado['CH07'].unique())
valores_unicos_p21 = np.sort(df_filtrado['P21'].unique())

# Imprimir los valores únicos ordenados
print("Valores únicos en REGION:", valores_unicos_region)
print("Valores únicos en CH04:", valores_unicos_ch04)
print("Valores únicos en NIVEL_ED:", valores_unicos_nivel_ed)
print("Valores únicos en CH07:", valores_unicos_ch07)
print("Valores únicos en P21:", valores_unicos_p21)

Valores únicos en REGION: [ 1 40 41 42 43 44]
Valores únicos en CH04: [1 2]
Valores únicos en NIVEL_ED: [1 2 3 4 5 6 7]
Valores únicos en CH07: [1 2 3 4 5]
Valores únicos en P21: [      0    1000    1200    1500    2000    3000    4000    4400    4500
    4800    5000    5600    6000    6400    7000    7500    8000    9000
   10000   10500   11000   12000   12400   14000   15000   15400   15600
   16000   16800   17000   18000   18500   18600   19000   19200   20000
   20400   20800   21000   22000   22400   23000   24000   25000   25200
   25600   26000   27000   28000   28320   28800   29000   30000   31000
   31200   32000   32400   33000   34000   34800   35000   35100   35200
   36000   37500   38000   38400   40000   42000   43000   43200   43500
   44000   45000   46000   46800   47000   48000   49000   50000   51000
   52000   52500   52700   52800   53000   54000   54700   55000   56000
   57000   57600   57700   58000   59000   60000   60800   61000   62000
   62400   63000  

In [37]:
# Calcular el logaritmo natural de P21 (ingreso) en el DataFrame filtrado
df_filtrado['log_P21'] = np.log(df_filtrado['P21'] + 1)  # Sumar 1 para evitar log(0)

# Convertir las variables categóricas a tipo 'category' en el DataFrame filtrado
df_filtrado['REGION'] = df_filtrado['REGION'].astype('category')
df_filtrado['CH04'] = df_filtrado['CH04'].astype('category')
df_filtrado['NIVEL_ED'] = df_filtrado['NIVEL_ED'].astype('category')
df_filtrado['CH07'] = df_filtrado['CH07'].astype('category')

# Seleccionar las variables de interés en el DataFrame filtrado
variables = ['REGION', 'CH04', 'NIVEL_ED', 'CH07', 'log_P21']
df_filtrado = df_filtrado[variables].dropna()  # Eliminar filas con valores faltantes

# 2. Especificar el modelo usando R-style formulas
formula = 'log_P21 ~ C(REGION) + C(CH04) + C(NIVEL_ED) + C(CH07)'

# 3. Ajustar el modelo
modelo = smf.ols(formula, data=df_filtrado).fit()

# 4. Mostrar los resultados
print(modelo.summary())

KeyError: 'P21'